Code to make an SQLite database from ABCD data download

https://www.sqlitetutorial.net/sqlite-python/creating-tables/

In [132]:
import sqlite3
import os 
from sqlite3 import Error
from pathlib import Path
import pandas as pd

In [133]:
def create_connection(db_file):
    """ create a database connection to a SQLite database 
    params:
        db_file: 
            database file name to be connected

    return:
        connection object or None
    """
    
    # start database as empty text file
    Path(db_file).touch()


    # create sqlite table
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)

    return conn

# Usage 
# create_connection(r"./data/test.db")

In [134]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    params:
        conn: 
            Connection object
        create_table_sql: 
            a CREATE TABLE statement

    returns:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [135]:
def get_txt_data(file_name):
    """ creates pandas dataframe containing data values 
    
    params:

    return:
        pandas dataframe containing data
    
    """


    # Create table from txt file
    table = pd.read_csv('./data/data-files/{}.txt'.format(file_name), delimiter='\t')
    col_desc = table.loc[0].values  # get column descriptions
    table.drop(labels=0, inplace=True)    # remove column descriptions

    return table

# Usage
#file_name = "abcd_bp01"
#data_table = get_txt_data(file_name)
#data_table.head()


In [136]:
def get_csv_info(file_name):
    """ creates pandas dataframe containing dataset information/descriptions from csv files 
    
    params:

    return:
        pandas dataframe containing data information
    
    """

    # Create table from csv file
    table = pd.read_csv('./data/abcd-4.0-data-dictionaries/{}.csv'.format(file_name))

    # convert to sql data types
    table.replace({'GUID': 'TEXT', 'String': 'TEXT', 'Date': 'TEXT', 'Integer': 'INTEGER', 'Float': 'REAL'}, inplace=True)    # replace DataType with SQLlite data types

    return table

# Usage
#file_name = "abcd_bp01"
#info_table = get_csv_info(file_name)
#info_table.head()
    

In [182]:
def new_col_entry(column_name, data_type, comment='', constraint='', last_elem=False):
    """ creates entry for sqlite table 
    
    params:

    return:
        str with entry to put in CREATE TABLE command 
    
    """
    entry = '\t\t\t\t {} {} {}, \t -- {}\n'

    # no comma for last element
    if last_elem:
        entry = '\t\t\t\t {} {} {} \t -- {}\n'

    return entry.format(column_name, data_type, constraint, comment)


In [233]:
def sql_create_table_cmd(file_name):
    """ constructs CREATE TABLE entry for sqlite table using abcd file
    
    params:
        file_name:

    return:
        str with sqlite CREATE TABLE command
    """

    # open CREATE TABLE command
    sql_cmd="""CREATE TABLE IF NOT EXISTS {} (\n""".format(file_name)

    # Manually add in first 3 ids from txt file (common to all .txts)
    sql_cmd += new_col_entry('collection_id', 'TEXT', 'ID associated with data collection')
    sql_cmd += new_col_entry('{}_id'.format(file_name), 'TEXT', 'ID associated with data.txt file (experiment)')
    sql_cmd += new_col_entry('dataset_id', 'TEXT', 'ID associated with dataset')

    # Get column info
    info_table = get_csv_info(file_name)
    
    # Add remaining data column entries
    for index, row in info_table.iterrows():
        if row['ElementName'] == 'subjectkey':
            sql_cmd += new_col_entry(row['ElementName'], row['DataType'], row['ElementDescription'], constraint='PRIMARY KEY')
            print(index, row['ElementName'])

        #elif index == len(info_table.index) - 1:
            #sql_cmd += new_col_entry(row['ElementName'], row['DataType'], row['ElementDescription'], last_elem=True)
            #print(index, row['ElementName'])

        else:
            sql_cmd += new_col_entry(row['ElementName'], row['DataType'], row['ElementDescription'])
            print(index, row['ElementName'])


    # close CREATE TABLE command
    sql_cmd += """\t\t\t\t);"""     # close command


    return sql_cmd

# Usage
#file_name = "abcd_bp01"
#sql_create_table_cmd(file_name)

In [234]:
def main():
    # Change to ABCD directory
    os.chdir(r"/Users/margotwagner/ucsd/research/ABCD")
    
    # Name database
    database = r"./data/abcd.db"

    # Get list of all data file name
    data_file_names = [f.split('.')[0] for f in os.listdir('./data/data-files')]
    info_file_names = [f.split('.')[0] for f in os.listdir('./data/abcd-4.0-data-dictionaries')]

    # Only want files with data and info (for now)
    files_data_only = set(data_file_names) - set(info_file_names)   # file names with txt data but no csv info
    files_info_only = set(info_file_names) - set(data_file_names)   # file names with csv info but no txt data

    file_names = data_file_names.copy()
    for f in files_data_only:
        file_names.remove(f)

    print(len(file_names))

    # sqlite statements
    file_name = "abcd_bp01"
    sql_cmd = sql_create_table_cmd(file_name)
    print(sql_cmd)

    #sql_cmd = """CREATE TABLE IF NOT EXISTS abcd_bp01 (
	#			 collection_id TEXT , 	 -- ID associated with data collection
	#			 abcd_bp01_id TEXT , 	 -- ID associated with data.txt file (experiment)
	#			 dataset_id TEXT  	 -- ID associated with dataset
	#			); """ 

    #print(sql_cmd)

    
    #sql_create_projects_table = """ CREATE TABLE IF NOT EXISTS projects (
    #                                    id integer PRIMARY KEY,
    #                                    name text NOT NULL,
    #                                    begin_date text,
    #                                    end_date text
    #                                ); """

    # sql_create_tasks_table = """CREATE TABLE IF NOT EXISTS tasks (
    #                                id integer PRIMARY KEY,
    #                                name text NOT NULL,
    #                                priority integer,
    #                                status_id integer NOT NULL,
    #                                project_id integer NOT NULL,
    #                                begin_date text NOT NULL,
    #                                end_date text NOT NULL,
    #                                FOREIGN KEY (project_id) REFERENCES projects (id)
    #                            );"""

    # create a database connection
    conn = create_connection(database)

    # create tables
    if conn is not None:
        # create abcd_bp01 table
        create_table(conn, sql_cmd)

        # create tasks table
        #create_table(conn, sql_create_tasks_table)
    else:
        print("Error! cannot create the database connection.")

if __name__ == '__main__':
    main()


359
0 subjectkey
1 src_subject_id
2 interview_date
3 interview_age
4 sex
5 eventname
6 blood_pressure_start_time
7 blood_pressure_refuse
8 blood_pressure_test_sys_1
9 blood_pressure_test_dia_1
10 blood_pressure_pulse_1
11 blood_pressure_test_sys_2
12 blood_pressure_test_dia_2
13 blood_pressure_pulse_2
14 blood_pressure_test_sys_3
15 blood_pressure_test_dia_3
16 blood_pressure_pulse_3
17 blood_pressure_sys_mean
18 blood_pressure_dia_mean
CREATE TABLE IF NOT EXISTS abcd_bp01 (
				 collection_id TEXT , 	 -- ID associated with data collection
				 abcd_bp01_id TEXT , 	 -- ID associated with data.txt file (experiment)
				 dataset_id TEXT , 	 -- ID associated with dataset
				 subjectkey TEXT PRIMARY KEY, 	 -- The NDAR Global Unique Identifier (GUID) for research subject
				 src_subject_id TEXT , 	 -- Subject ID how it's defined in lab/project
				 interview_date TEXT , 	 -- Date on which the interview/genetic test/sampling/imaging/biospecimen was completed. MM/DD/YYYY
				 interview_age 